In [1]:
from dataloaders import data_set,data_dict
import torch
import yaml
import os
import torch.nn as nn

import torch
import torch.optim as optim

from dataloaders import data_set,data_dict
from models.model_builder import model_builder
import pandas as pd

import numpy as np

import ray
from ray import tune, air
from ray.air import session, Checkpoint
from ray.tune.examples.mnist_pytorch import ConvNet, get_data_loaders, test
from ray.tune.schedulers import PopulationBasedTraining

import random
from ray.tune.schedulers.pb2 import PB2

from pbtsearch import train

from analysis import cal_test_accuracy

/home/janis/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-08-18 20:24:42,329	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-18 20:24:42,376	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-08-18 20:24:42,452	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
config_path = os.path.abspath('configs/data.yaml')
model_config = os.path.abspath('configs/model.yaml')
root_path = os.path.abspath("datasets")
to_save_path = os.path.abspath("ISWC2022LearnableFilter/Run_logs")   
freq_save_path = os.path.abspath("ISWC2022LearnableFilter/Freq_data")
window_save_path = os.path.abspath("ISWC2022LearnableFilter/Sliding_window")

class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    def __getattr__(self, attr):
        if attr.startswith('__'):
            raise AttributeError("Should not start with __")
        return self.get(attr)
    #__getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

def get_args():
    args = dotdict()   
    
    
    args.training_iterations = 100
    args.restore = False
    args.perturbation_interval = 3
    args.trials = 16
    args.scheduler = "pbt"
    args.storage_path = "/home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt"
    args.cpu_per_trial = 1
    args.gpu_per_trial = 0.2
    args.synch = True
    #exploit 
    args.quantile_fraction = 0.25
    args.trainable_api = 'class'
    args.reuse_actor = True
    
    #only pbt
    args.resample_probability = 0.2
    args.burn_in_period = 0
    

    args.seed = 1

    args.to_save_path     = to_save_path
    args.freq_save_path   = freq_save_path
    args.window_save_path = window_save_path
    args.root_path        = root_path

    args.drop_transition  = False
    args.datanorm_type    = "standardization" # None ,"standardization", "minmax"

    args.batch_size       = 256                                                    
    args.shuffle          = True
    args.drop_last        = False
    args.train_vali_quote = 0.90                                           


    # training setting 
    #args.train_epochs            = 150

    args.learning_rate           = 0.001  
    args.learning_rate_patience  = 5
    args.learning_rate_factor    = 0.1


    args.early_stop_patience     = 15

    #args.use_gpu                 = True if torch.cuda.is_available() else False
    #args.gpu                     = 0
    #args.use_multi_gpu           = False

    args.optimizer               = "Adam"
    args.criterion               = "CrossEntropy"

    args.data_name                        =  "hapt"
    args.use_vali_keys = True
    

    args.wavelet_filtering                = False
    args.wavelet_filtering_regularization = False
    args.wavelet_filtering_finetuning     = False


    args.difference       = False 
    args.filtering        = False
    args.magnitude        = False
    args.weighted_sampler = False

    args.pos_select       = None
    args.sensor_select    = None


    args.representation_type = "time"
    #args.exp_mode            = "LOCV"
    args.exp_mode            = "Given"

  #  if args.data_name      ==  "skodar":
  #      args.exp_mode            = "SOCV"

    config_file = open(config_path, mode='r')
    data_config = yaml.load(config_file, Loader=yaml.FullLoader)
    config = data_config[args.data_name]

    args.root_path       = os.path.join(args.root_path,config["filename"])
    args.sampling_freq   = config["sampling_freq"]
    args.num_classes     =  config["num_classes"]
    window_seconds       = config["window_seconds"]
    args.windowsize      =   int(window_seconds * args.sampling_freq) 
    args.input_length    =  args.windowsize
    # input information
    args.c_in            = config["num_channels"]
    
    if args.difference:
        args.c_in  = args.c_in * 2
    if  args.filtering :
        for col in config["sensors"]:
            if "acc" in col:
                args.c_in = args.c_in+1

    if args.wavelet_filtering :
        
        if args.windowsize%2==1:
            N_ds = int(torch.log2(torch.tensor(args.windowsize-1)).floor()) - 2
        else:
            N_ds = int(torch.log2(torch.tensor(args.windowsize)).floor()) - 2

        args.f_in            =  args.number_wavelet_filtering*N_ds+1
    else:
        args.f_in            =  1


    args.filter_scaling_factor = 0.25
    args.model_type            = "deepconvlstm"
    args.model_config = model_config

    args.mixup_alpha  = 0.4
    args.mixup_argmax = True

    args.max_aug = 3

    args.random_augmentation_config = {"jitter":True,
                                    "moving_average":True,
                                    "magnitude_scaling":True,
                                    "magnitude_warp":True,
                                    "magnitude_shift":True,
                                    "time_warp":True,
                                    "window_warp":True,
                                    "window_slice":True,
                                    "random_sampling":True,
                                    "slope_adding":True
                                    }
    random_augmentation_nr = 0
    for key in args.random_augmentation_config.keys():
        if args.random_augmentation_config[key]==1:
            random_augmentation_nr = random_augmentation_nr+1
    args.random_augmentation_nr = random_augmentation_nr

    return args

In [3]:
args = get_args()
result_grid = train(args)

(RayModel pid=78841) Train data number :  8745
(RayModel pid=78841) The number of classes is :  12
(RayModel pid=78841) The input_length  is :  128
(RayModel pid=78841) The channel_in is :  6
(RayModel pid=78841) Validation data number :  1723
(RayModel pid=78841) Build the DeepConvLSTM model!
(RayModel pid=78910) new best score!!!!
(RayModel pid=78909) Train data number :  8745 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/ray-logging.html#log-deduplication for more options.)
(RayModel pid=78909) The number of classes is :  12 [repeated 4x across cluster]
(RayModel pid=78909) The input_length  is :  128 [repeated 4x across cluster]
(RayModel pid=78909) The channel_in is :  6 [repeated 4x across cluster]
(RayModel pid=78909) Validation data number :  1723 [repeated 4x across cluster]
(RayModel pid=78909) Build the DeepConvLSTM model! [repeated 4x across cluster]
(

2023-08-18 20:25:15,660	WARNING tune.py:192 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 


(RayModel pid=78910) new best score!!!! [repeated 5x across cluster]


2023-08-18 20:25:18,684	ERROR tune.py:1144 -- Trials did not complete: [RayModel_8427d_00000, RayModel_8427d_00001, RayModel_8427d_00002, RayModel_8427d_00003, RayModel_8427d_00004]
2023-08-18 20:25:18,684	INFO tune.py:1148 -- Total run time: 29.33 seconds (26.30 seconds for the tuning loop).
2023-08-18 20:25:18,685	WARNING tune.py:1163 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt/RayModel_2023-08-18_20-24-44", trainable=...)
2023-08-18 20:25:18,699	WARNING experiment_analysis.py:916 -- Failed to read the results for 11 trials:
- /home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt/RayModel_2023-08-18_20-24-44/RayModel_8427d_00005_5_mixup_probability=0.5000,random_aug_first=True,random_augmentation_prob=1.0000_2023-08-18_20-24-49
- /home/janis/PopulationBasedTraining/RayTuneResults/hapt/pbt/RayModel_2023-08-18_20-24-44/RayModel_8427d_00006_6_mixup_p

In [ ]:
best_result = result_grid.get_best_result(mode="max", metric="mean_accuracy")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

checkpoint = best_result.checkpoint
if args.trainable_api=='function':
    dict = checkpoint.to_dict()
elif args.trainable_api=='class':
    dict = torch.load(checkpoint._local_path+'/model.pht')
else:
    raise AttributeError()

total_loss,  acc, f_w,  f_macro, f_micro = cal_test_accuracy(args, device, dict)
text = 'Best Trial Test Performance: Total_Loss {} Acc {}  F_w {} F_Macro {} F_Micro {}'.format(total_loss,  acc, f_w,  f_macro, f_micro)
print(text)

2023-08-18 18:27:15,989	WARNING experiment_analysis.py:783 -- Could not find best trial. Did you pass the correct `metric` parameter?


RuntimeError: No best trial found for the given metric: mean_accuracy. This means that no trial has reported this metric, or all values reported for this metric are NaN. To not ignore NaN values, you can set the `filter_nan_and_inf` arg to False.